In [1]:
from __future__ import print_function
from __future__ import absolute_import

In [2]:
import sys
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

/home/sracela/miniconda3/envs/p2c/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sracela/miniconda3/envs/p2c/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sracela/miniconda3/envs/p2c/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sracela/miniconda3/envs/p2c/lib/python3.6/s

In [3]:
from classes.dataset.Generator import *
# from classes.model.pix2code_attention import *

### Images and captions

Transformed images in training dataset to numpy arrays already provided in 

../datasets/web/training_features

Normalized pixel values and resized pictures 

In [4]:
# Store captions and image names in vectors
# Preprocess the images && Preprocess and tokenize the captions

input_path = '../datasets/android/training_features'
output_path = '../bin'

In [5]:
# Load caption annotation files and image files in Dataset
dataset = Dataset()
dataset.load(input_path, generate_binary_sequences=True)
dataset.save_metadata(output_path)
dataset.voc.save(output_path)

Loading data...
Generating sparse vectors...
Dataset size: 86150
Vocabulary size: 20
Input shape: (256, 256, 3)
Output size: 20
Maz length: 104, element 61643
['<START>', 'stack', '{', '\n', 'row', '{', '\n', 'switch', ',', 'btn', ',', 'btn', ',', 'switch', '\n', '}', '\n', 'row', '{', '\n', 'switch', ',', 'switch', ',', 'switch', ',', 'switch', '\n', '}', '\n', 'row', '{', '\n', 'label', ',', 'slider', ',', 'label', '\n', '}', '\n', 'row', '{', '\n', 'label', ',', 'btn', '\n', '}', '\n', 'row', '{', '\n', 'label', ',', 'slider', ',', 'label', '\n', '}', '\n', 'row', '{', '\n', 'switch', ',', 'btn', ',', 'switch', '\n', '}', '\n', 'row', '{', '\n', 'label', ',', 'btn', '\n', '}', '\n', 'row', '{', '\n', 'label', ',', 'slider', ',', 'label', '\n', '}', '\n', '}', '\n', 'footer', '{', '\n', 'btn-notifications', ',', 'btn-dashboard', '\n', '}', '\n', '<END>']


In [ ]:
len(dataset.partial_sequences[5])


In [ ]:
# USANDO GENERATOR

gui_paths, img_paths = Dataset.load_paths_only(input_path)

In [ ]:
input_shape = dataset.input_shape
output_size = dataset.output_size
steps_per_epoch = dataset.size / BATCH_SIZE

In [ ]:
voc = Vocabulary()
voc.retrieve(output_path)

In [ ]:

generator = Generator.data_generator(voc, gui_paths, img_paths, batch_size=BATCH_SIZE, generate_binary_sequences=True)

In [ ]:
CONTEXT_LENGTH, BATCH_SIZE, input_shape

# Model

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import concatenate

## Image model

In [ ]:
def image_model_sequential(input_shape):
    # initialize the model along with the input shape to be
    # "channels last" ordering
    model = Sequential()
    # define CONV => RELU => MAXPOOL => DROPOUT block
    model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # classifier
    model.add(Flatten())
    # para attention!
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.3))

    # repeat the structure CONTEXT_LENGHT times (one image per context vector) 
    # repeat_vector_1 = (None,48,1024)
    model.add(RepeatVector(CONTEXT_LENGTH))

    # return the constructed network architecture
    return model

In [ ]:
image_model = image_model_sequential(input_shape) 

In [ ]:
image_model.summary()

In [ ]:
from tensorflow.keras.layers import LSTM
#from keras.layers import LSTM

## Language model

In [ ]:
def language_model_sequential(output_size):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(CONTEXT_LENGTH, output_size)))
    model.add(LSTM(128, return_sequences=True))
    return model

In [ ]:
language_model = language_model_sequential(output_size) 

In [ ]:
language_model.summary()

## Pix2Code Model
- Define inputs
- Define image and language models
- Call image and language models with their respective inputs
- Concantenate them
- Call decoder with this previous concatenation
- Softmax

In [ ]:
def pix2code(input_shape, output_size):
    # define inputs
    visual_input = Input(shape=input_shape)        
    textual_input = Input(shape=(CONTEXT_LENGTH, output_size))    
    
    image_model = image_model_sequential(input_shape)
    language_model = language_model_sequential(output_size)
    
    encoded_image = image_model(visual_input)
    encoded_text = language_model(textual_input)            

    decoder = concatenate([encoded_image, encoded_text])

    decoder = LSTM(512, return_sequences=True)(decoder)
    decoder = LSTM(512, return_sequences=False)(decoder)
    
    decoder = Dense(output_size, activation='softmax')(decoder)
    
    model = Model(inputs=[visual_input, textual_input], outputs=decoder)
    return model
        

### ... WITH ATTENTION

In [ ]:
def pix2code_attention(input_shape, output_size):
    # define inputs
    visual_input = Input(shape=input_shape)        
    textual_input = Input(shape=(CONTEXT_LENGTH, output_size))    
    
    image_model = image_model_sequential(input_shape)
    language_model = language_model_sequential(output_size)
    
    encoded_image = image_model(visual_input)
    encoded_text = language_model(textual_input)
    
    # Encoder

    encoder_out = concatenate([encoded_image, encoded_text])
    
    # Decoder

    decoder_out= LSTM(128, return_sequences=True)(encoder_out)
    
    #Attention
    query_value_attention_seq = tf.keras.layers.Attention()([encoded_text, decoder_out])
    
    #decoder_out = tf.keras.layers.GlobalAveragePooling1D()(decoder_out)
    #attn_out = tf.keras.layers.GlobalAveragePooling1D()(query_value_attention_seq)
    
    # Concat attention input and decoder GRU output
    decoder_concat_input = concatenate([decoder_out, query_value_attention_seq])
    
    decoder = LSTM(128, return_sequences=False)(decoder_concat_input)
    
    decoder = Dense(output_size, activation='softmax')(decoder)
    
    model_attention = Model(inputs=[visual_input, textual_input], outputs=decoder)
    return model_attention

In [ ]:
attention_model = pix2code_attention(input_shape, output_size)

In [ ]:
attention_model.summary()

## Train the model

In [ ]:
# instantiate a Keras Model sub-class model
model = pix2code(input_shape, output_size)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
#from keras.optimizers import RMSprop
#from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [ ]:
# initialize the optimizer and compile the model
optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
print("[INFO] training network...")
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
attention_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
tensorboard = TensorBoard(
    log_dir= "{}/logs".format(output_path),
    histogram_freq=0,
    write_images=True
    )
keras_callbacks = [
    tensorboard,
    EarlyStopping(monitor='loss', patience=5, mode='min', min_delta=0.0001),
    ModelCheckpoint("{}/checkpoints/prueba_test.h5".format(output_path), monitor='loss', save_best_only=True, mode='min')
    ]

In [ ]:
# train the network
seq_Model = model.fit_generator(generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=keras_callbacks)

In [ ]:
att_Model = attention_model.fit_generator(generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=keras_callbacks)

In [ ]:
name = "jupyterp2c_att"

In [ ]:
model_json = model.to_json()
with open("{}/{}.json".format(output_path, name), "w") as json_file:
    json_file.write(model_json)
model.save_weights("{}/{}.h5".format(output_path, name))

In [ ]:
model_json = attention_model.to_json()
with open("{}/{}.json".format(output_path, name), "w") as json_file:
    json_file.write(model_json)
attention_model.save_weights("{}/{}.h5".format(output_path, name))

# Evaluate the model

In [ ]:
from classes.Sampler import *

In [ ]:
output_path, name

In [ ]:
with open("{}/{}.json".format(output_path, name), "r") as json_file:
            loaded_model_json = json_file.read()
model2 = tf.keras.models.model_from_json(loaded_model_json, custom_objects={'Attention': tf.keras.layers.Attention})
model2.load_weights("{}/{}.h5".format(output_path, name))

In [ ]:
sampler = Sampler(output_path, input_shape, output_size, CONTEXT_LENGTH)

In [ ]:
eval_path = '../datasets/android/eval_set/images/'
beam_width = 3
results_path = "../code"

In [ ]:
for f in os.listdir(eval_path):
    if f.find(".png") != -1:
        evaluation_img = Utils.get_preprocessed_img("{}/{}".format(eval_path, f), IMAGE_SIZE)
        file_name = f[:f.find(".png")]
        print(file_name)
        print("Search with beam width: {}".format(beam_width))
        result, _ = sampler.predict_beam_search(model2, np.array([evaluation_img]), beam_width=beam_width)
        print("Result beam: {}".format(result))
        with open("{}/jupyter/{}.gui".format(results_path, file_name), 'w') as out_f:
            out_f.write(result.replace(START_TOKEN, "").replace(END_TOKEN, ""))

## Show results

- Modify compile_batch.sh (code/jupyter/*)
- Run compile_bash.sh
- Split jupyter in screens, xmls, guis
- copy xmlls in android/Randomizer/app/src/main/res/layout/screens/
- Run layout2image.sh